# Combining Datasets with Pandas

## Learning Objectives

At the end of this notebook you should be able to
- combine DataFrames with Pandas
- describe the different joining methods (how to join DataFrames)

Pandas functions that allow us to combine two sets of data include the use of `pd.merge()`, `df.join()`, `df.merge()`, and `pd.concat()`. For the most part, these do largely the same things (although you'll notice the slight syntax difference with `merge()` and `concat()` being able to be called via the Pandas module and `merge()` and `join()` being able to be called on a DataFrame instance). There are some cases where one of these might be better than another in terms of writing less code or performing some kind of data combination in an easier way. The major differences between these, though, largely depend on what they do by default when you try to combine different data. By default, `merge()` looks to join on common columns, `join()` on common indices, and `concat()` by just appending on a given axis.

You can find more detail about the differences between all three of these in the [docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html). We'll look at some examples below. 

In [ ]:
# We'll go back to our wine data set. Who doesn't love wine?
import pandas as pd
wine_df = pd.read_csv('data/winequality-red.csv', delimiter=';')
wine_df.head()

In [ ]:
# A glance at the values of the quality of wine in the DataFrame
wine_df.quality.unique()

In [ ]:
# get_dummies is a method called on the pandas module - you simply pass in a Pandas Series 
# or DataFrame, and it will convert a categorical variable into dummy/indicator variables. 
# The idea of dummy coding is to convert each category into a new column, and assign a 1 or 0 to the column.
# We will cover this topic in more depth later.
quality_dummies = pd.get_dummies(wine_df.quality, prefix='quality')
quality_dummies.head()

### Join()
Now let's look at the `join()` method. Remember, this joins on indices by default. This means that we can simply join our quality dummies dataframe back to our original wine dataframe with the following...

In [ ]:
joined_df = wine_df.join(quality_dummies)
joined_df.head() 

The arguments of `.join` are the following: 
````
DataFrame.join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
````
With `how` we can specify which join method we want to use.

The how argument to merge specifies how to determine which keys are to be included in the resulting table. If a key combination does not appear in either the left or right tables, the values in the joined table will be NA. Here is a summary of the how options and their SQL equivalent names:

Merge method | SQL Join Name | Description
---|---|---
left| LEFT OUTER JOIN | Use keys from left frame only
right | RIGHT OUTER JOIN | Use keys from right frame only
outer | FULL OUTER JOIN | Use union of keys from both frames
inner | INNER JOIN | Use intersection of keys from both frames


You can also think of it as set theory and use Venn diagrams to illustrate what happens in each method.

![Join Methods](./images/join_types.png)

### Concat()
Let's now look at concat.
Similar to `join` we can  specify the method we want to use for combining the datasets.

Different from join and merge, which by default operate on columns, concat can define whether to operate on columns or rows.
In the images below, you can see the differences, if axis is set as 0 or 1.

**Concat with axis=0:**
![Concat Axis 0](./images/concat_axis_0.png)

---

**Concat with axis=1:**
![Concat Axis 1](./images/concat_axis_1.png)

(The pictures were part of [this](https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78) blog post.)

In [ ]:
joined_df2 = pd.concat([quality_dummies, wine_df], axis=1)
joined_df2.head()

Let's read in a different data set, since we're looking at combining multiple data sources.

In [ ]:
red_wines_df = pd.read_csv('data/winequality-red.csv', delimiter=';')
white_wines_df = pd.read_csv('data/winequality-white.csv', delimiter=';')

In [ ]:
red_wines_df.columns

In [ ]:
white_wines_df.columns

In [ ]:
red_wines_quality_df = red_wines_df.groupby('quality').mean()['fixed acidity'].reset_index()
red_wines_quality_df.head()

In [ ]:
white_wines_quality_df = white_wines_df.groupby('quality').mean()['fixed acidity'].reset_index()
white_wines_quality_df.head()

In [ ]:
pd.merge(red_wines_quality_df, white_wines_quality_df, on=['quality'], suffixes=[' red', ' white'])

Please try out to generate the table above using the methods `.join()` and `.concat()`.

## Check your understanding

1. Please join the two given dataframes (df1 and df2) along rows and merge with the third (df3) dataframe along the common column id.


In [ ]:
df1 = pd.DataFrame({
        'student_id': ['S1', 'S2', 'S3', 'S4', 'S5'],
         'name': ['Erika Raaf', 'Nadja Berens', 'Florentin Kleist', 'Dorothea Eibl', 'Gerhard Bihlmeier'], 
        'subject': ['Math', 'Biology', 'Biology', 'English', 'Philosophy']})
df2 = pd.DataFrame({
        'student_id': ['S4', 'S5', 'S6', 'S7', 'S8'],
        'name': ['Franz Xaver Schild', 'Ann-Kathrin Heiny', 'Jens Hüls', 'Vera Kagan', 'Paula Brodersen'], 
        'subject': ['Chemistry', 'Economics', 'Math', 'Math', 'Social Science']})
df3 = pd.DataFrame({
        'student_id': ['S1', 'S2', 'S3', 'S4', 'S5', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13'],
        'marks': [23, 45, 12, 67, 21, 55, 33, 14, 56, 83, 88, 12]})

```Python
result_data = pd.concat([df1, df2])
final_merged_data = pd.merge(result_data, df3, on='student_id')
final_merged_data
```


2. You have received some weather data (temperature) of the last year. For each month the average temperature was measured, only for a few months the maximum temperature could be measured. Anyway, you want to combine these two data without losing any information.

(Extra question: Can you fill in the average max. Temperature for the missing values in the Column `Max TemperatureF`)

In [ ]:
weather_mean_data = {'Mean TemperatureF': [53.1, 70., 34.93548387, 28.71428571, 32.35483871, 72.87096774, 70.13333333, 35., 62.61290323, 39.8, 55.4516129 , 63.76666667],
                     'Month': ['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep']}
weather_max_data = {'Max TemperatureF': [68, 89, 91, 84], 'Month': ['Jan', 'Apr', 'Jul', 'Oct']}
